# Create a FABRIC Facility Port

This notebook shows how to use create an facility port to connect your FABRIC experiment to an external facility. 



## Import the FABlib Library

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,1ecd9d6a-7701-40fa-b78e-b2293c9526ed
Bastion Host,bastion.fabric-testbed.net
Bastion Username,sw_slcolson_0000101181
Bastion Private Key File,/home/fabric/work/fabric_config/fabric-bastion-key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Sites to avoid,


## Create the Experiment Slice

The following creates a single node with basic compute capabilities. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated.   

By default, the submit function will block until the node is ready and will display the progress of your slice being built.



In [2]:
slice_name = "Slice-AWSDX"

facility_port_site='FIU'

### Facility port info

#### Find the facility ports on the site
List the facility ports
- Facility Port Name
- Available VLAN range
- Currently allocated VLAN range

In [3]:
cell_output = 'pandas' # 'json'
output_list = fablib.list_facility_ports(filter_function=lambda x: x['site_name'] == facility_port_site, output=cell_output)

Name,Site,Interface Name,VLAN Range,Allocated VLAN Range,Local Name,Device Name,Region
AmLight-EXP-Layer2-FIU,FIU,port+fiu-data-sw:HundredGigE0/0/0/23:facility+AmLight-EXP-Layer2-FIU-int,['4000-4019'],N/A,N/A,N/A,N/A
AmLight-Layer3-FIU,FIU,port+fiu-data-sw:HundredGigE0/0/0/23:facility+AmLight-Layer3-FIU-int,['3001-3008'],N/A,N/A,N/A,N/A


#### Select the Facility Port on the site

Set the facility port name and select the VLAN to use

In [4]:
facility_port_name = 'AmLight-EXP-Layer2-FIU' # VLAN Range: 4000-4019
#facility_port_name ='AmLight-Layer3-FIU'     # VLAN Range: 3001-3008

facility_port_vlan='4015'

### Create Slice

Create new_slice and add node(s) to the FABRIC topology

In [5]:
slice = fablib.new_slice(name=slice_name)

# Example: One VM on FABRIC-STAR with a NIC - sharedNIC (basic) or dedicated smartNIC (ConnectX_6)
node = slice.add_node(name=f"Node1", site='STAR')

#node_iface = node.add_component(model='NIC_ConnectX_6', name="nic1").get_interfaces()[0]
node_iface = node.add_component(model='NIC_Basic', name="nic1").get_interfaces()[0]

Add facility port to the network service configuration along with the node interface(s)

In [6]:
facility_port = slice.add_facility_port(name=facility_port_name, site=facility_port_site, vlan=facility_port_vlan)
facility_port_interface =facility_port.get_interfaces()[0]

#net = slice.add_l2network(name=f'net_facility_port', interfaces=[node_iface,facility_port_interface])
net = slice.add_l2network(name=f'net_facility_port', interfaces=[])
net.add_interface(node_iface)
net.add_interface(facility_port_interface)

In [7]:
print(f"facility_port.get_site(): {facility_port.get_site()}")
print(f"facility_port.get_fim_interface(): {facility_port.get_fim_interface()}")

facility_port.get_site(): FIU
facility_port.get_fim_interface(): {'node_id': '1fb977b1-2c8e-4eee-b64a-46f38815ec5a', 'name': 'AmLight-EXP-Layer2-FIU', 'site': 'FIU', 'type': 'Facility'}


Submit slice request to create

In [8]:
slice.submit();


Retry: 8, Time: 248 sec


ID,488dad25-f5fc-401b-a2df-212dd499514d
Name,Slice-AWSDX
Lease Expiration (UTC),2024-10-24 11:26:44 +0000
Lease Start (UTC),2024-10-23 11:26:44 +0000
Project ID,1ecd9d6a-7701-40fa-b78e-b2293c9526ed
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
578eea53-0109-4817-a6b9-9452046c7b28,Node1,2,8,10,default_rocky_8,qcow2,star-w5.fabric-testbed.net,STAR,rocky,2001:400:a100:3030:f816:3eff:fe90:e999,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3030:f816:3eff:fe90:e999,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
a7c7a4ef-48ef-4a9c-98ff-d89f76ccf063,net_facility_port,L2,L2STS,None,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
Node1-nic1-p1,p1,Node1,net_facility_port,100,config,,06:C2:F3:99:A2:F2,eth1,eth1,fe80::7522:9541:2884:493b,4,HundredGigE0/0/0/1



Time to print interfaces 252 seconds


## Observe the Slice's Attributes


In [9]:
slice = fablib.get_slice(slice_name)

In [10]:
slice.show()
slice.list_nodes()
slice.list_networks()
slice.list_interfaces()


ID,488dad25-f5fc-401b-a2df-212dd499514d
Name,Slice-AWSDX
Lease Expiration (UTC),2024-10-24 11:26:44 +0000
Lease Start (UTC),2024-10-23 11:26:44 +0000
Project ID,1ecd9d6a-7701-40fa-b78e-b2293c9526ed
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
578eea53-0109-4817-a6b9-9452046c7b28,Node1,2,8,10,default_rocky_8,qcow2,star-w5.fabric-testbed.net,STAR,rocky,2001:400:a100:3030:f816:3eff:fe90:e999,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3030:f816:3eff:fe90:e999,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
a7c7a4ef-48ef-4a9c-98ff-d89f76ccf063,net_facility_port,L2,L2STS,None,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
Node1-nic1-p1,p1,Node1,net_facility_port,100,config,,06:C2:F3:99:A2:F2,eth1,eth1,fe80::7522:9541:2884:493b,4,HundredGigE0/0/0/1


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
Node1-nic1-p1,p1,Node1,net_facility_port,100,config,,06:C2:F3:99:A2:F2,eth1,eth1,fe80::7522:9541:2884:493b,4,HundredGigE0/0/0/1


## Run the Experiment

Most experiments will require automated configuration and execution. You can use the fablib library to execute arbitrary commands on your node. 

The following code demonstrates how to use fablib to execute a "Hello, FABRIC" bash script. The library uses the bastion and VM keys defined at the top of this notebook to jump through the bastion host and execute the script.

In [11]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

# Configure the interface(s) of the VM(s) with designated subnet
subnet = IPv4Network("192.168.1.0/24")
available_ips = list(subnet)[2:]

In [12]:
node1 = slice.get_node(name=f"Node1")        
node1_iface = node1.get_interface(network_name=f'net_facility_port') 
node1_addr = available_ips.pop(99)
print(f"node1_addr: {node1_addr}")
node1_iface.ip_addr_add(addr=node1_addr, subnet=subnet)

stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')

stdout, stderr = node1.execute(f'sudo ip link set dev {node1_iface.get_physical_os_interface_name()} up')

stdout, stderr = node1.execute(f'sudo ip link set dev {node1_iface.get_os_interface()} up')

node1_addr: 192.168.1.101
3: eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 06:c2:f3:99:a2:f2 brd ff:ff:ff:ff:ff:ff
    altname enp7s0
    inet 192.168.1.101/24 scope global eth1
       valid_lft forever preferred_lft forever
    inet6 fe80::7522:9541:2884:493b/64 scope link noprefixroute 
       valid_lft forever preferred_lft forever


In [13]:
# Ping the node(s) that are active on the Facility Port side (e.g 192.168.1.10)
node1 = slice.get_node(name=f"Node1")     
node1_iface = node1.get_interface(network_name=f'net_facility_port') 

stdout, stderr = node1.execute(f'ping -c 5 192.168.1.10')   # This is expected to fail. 

PING 192.168.1.10 (192.168.1.10) 56(84) bytes of data.
From 192.168.1.101 icmp_seq=1 Destination Host Unreachable
From 192.168.1.101 icmp_seq=2 Destination Host Unreachable
From 192.168.1.101 icmp_seq=3 Destination Host Unreachable
From 192.168.1.101 icmp_seq=4 Destination Host Unreachable
From 192.168.1.101 icmp_seq=5 Destination Host Unreachable

--- 192.168.1.10 ping statistics ---
5 packets transmitted, 0 received, +5 errors, 100% packet loss, time 4119ms
pipe 3


# Add SDXlib code here

In [1]:
import json
import pandas as pd
from pprint import pprint
import requests

from config import *
from sdxlib.sdx_client import SDXClient
from sdxlib.sdx_exception import SDXException

In [2]:
# Define the required attributes
url = "http://67.17.206.205:8080/SDX-Controller"

## List All Available Ports


In [3]:
topology_url = url + "/topology"
response = requests.get(topology_url)

port_list = []

if response.status_code == 200:
    data = response.json()
    
    topology_data = response.json()
    for node in topology_data['nodes']:
        for port in node['ports']:
            if port['status'] == "up" and port['nni'] == "":
                port_list.append(
                    {
                        "Port ID": port["id"],
                        "Status": port["status"]
                    }
                )
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")

available_ports = pd.DataFrame(port_list, index=None)
available_ports.style.hide(axis="index")

Port ID,Status
urn:sdx:port:ampath.net:Ampath3:50,up
urn:sdx:port:ampath.net:Ampath2:50,up
urn:sdx:port:ampath.net:Ampath1:50,up
urn:sdx:port:sax.net:Sax01:50,up
urn:sdx:port:sax.net:Sax02:50,up
urn:sdx:port:tenet.ac.za:Tenet02:50,up
urn:sdx:port:tenet.ac.za:Tenet03:50,up
urn:sdx:port:tenet.ac.za:Tenet01:50,up


## Create Client

In [4]:
client_name = "Test SDX SC24 L2VPN"
client_endpoints = [
    {"port_id": "urn:sdx:port:ampath.net:Ampath2:50", "vlan": "302"}, 
    {"port_id": "urn:sdx:port:tenet.ac.za:Tenet02:50", "vlan": "302"},
]

In [5]:
# Create the L2VPN object
client = SDXClient(url, client_name, client_endpoints)

In [6]:
try:
    response = client.create_l2vpn()  
    print("L2VPN creation successful!")
    pprint(response)
except SDXException as e:
    print(f"L2VPN creation failed: {e}") 

L2VPN creation successful!
{'service_id': '029cd862-e9c0-4300-872b-b4bcd856ed2e'}


In [7]:
l2vpn_data = client.get_all_l2vpns()

node_info_list = []

for service_id, service_details in l2vpn_data.items():
    service_dict = {}
    for key, value in service_details.items():
        service_dict[key] = value
    node_info_list.append(service_dict)

df = pd.DataFrame(node_info_list)
df.style.hide(axis="index")

service_id,name,endpoints,description,qos_metrics,notifications,ownership,creation_date,archived_date,status,state,counters_location,last_modified,current_path,oxp_service_ids
029cd862-e9c0-4300-872b-b4bcd856ed2e,Test SDX SC24 L2VPN,"[{'port_id': 'urn:sdx:port:ampath.net:Ampath2:50', 'vlan': '302'}, {'port_id': 'urn:sdx:port:tenet.ac.za:Tenet02:50', 'vlan': '302'}]",None,None,None,None,None,None,None,None,None,None,"[{'port_id': 'urn:sdx:port:ampath.net:Ampath2:50', 'vlan': '302'}, {'port_id': 'urn:sdx:port:ampath.net:Ampath2:40', 'vlan': '4095'}, {'port_id': 'urn:sdx:port:sax.net:Sax02:40', 'vlan': '4095'}, {'port_id': 'urn:sdx:port:sax.net:Sax02:41', 'vlan': '4094'}, {'port_id': 'urn:sdx:port:tenet.ac.za:Tenet02:41', 'vlan': '4093'}, {'port_id': 'urn:sdx:port:tenet.ac.za:Tenet02:50', 'vlan': '302'}]",None
0f8fa69c-68e8-48dc-ad0d-a46c1971af2c,demo 1 -- nni to nni,"[{'port_id': 'urn:sdx:port:ampath.net:Ampath1:40', 'vlan': '4090'}, {'port_id': 'urn:sdx:port:ampath.net:Ampath1:40', 'vlan': '4089'}, {'port_id': 'urn:sdx:port:sax.net:Sax01:40', 'vlan': '4087'}, {'port_id': 'urn:sdx:port:sax.net:Sax01:40', 'vlan': '4086'}]",None,None,None,None,None,None,None,None,None,None,"[{'port_id': 'urn:sdx:port:ampath.net:Ampath1:40', 'vlan': '4090'}, {'port_id': 'urn:sdx:port:ampath.net:Ampath1:40', 'vlan': '4089'}, {'port_id': 'urn:sdx:port:sax.net:Sax01:40', 'vlan': '4087'}, {'port_id': 'urn:sdx:port:sax.net:Sax01:40', 'vlan': '4086'}]",None
272e6b82-0883-47ea-986e-5c7afea271bb,demo 1,"[{'port_id': 'urn:sdx:port:ampath.net:Ampath3:50', 'vlan': '4094'}, {'port_id': 'urn:sdx:port:tenet.ac.za:Tenet02:41', 'vlan': '4095'}, {'port_id': 'urn:sdx:port:tenet.ac.za:Tenet02:41', 'vlan': '4094'}]",None,None,None,None,None,None,None,None,None,None,"[{'port_id': 'urn:sdx:port:ampath.net:Ampath3:50', 'vlan': '4094'}, {'port_id': 'urn:sdx:port:ampath.net:Ampath2:40', 'vlan': '4091'}, {'port_id': 'urn:sdx:port:sax.net:Sax02:40', 'vlan': '4091'}, {'port_id': 'urn:sdx:port:sax.net:Sax02:41', 'vlan': '4095'}, {'port_id': 'urn:sdx:port:tenet.ac.za:Tenet02:41', 'vlan': '4095'}, {'port_id': 'urn:sdx:port:tenet.ac.za:Tenet02:41', 'vlan': '4094'}]",None
4ab7ae01-a8b4-45c6-b7ed-82d9fecd1131,VLAN between AMPATH/300 and TENET/300,"[{'port_id': 'urn:sdx:port:ampath.net:Ampath3:50', 'vlan': '307'}, {'port_id': 'urn:sdx:port:tenet.ac.za:Tenet03:50', 'vlan': '4095'}]",None,None,None,None,None,None,None,None,None,None,"[{'port_id': 'urn:sdx:port:ampath.net:Ampath3:50', 'vlan': '307'}, {'port_id': 'urn:sdx:port:ampath.net:Ampath1:40', 'vlan': '4095'}, {'port_id': 'urn:sdx:port:sax.net:Sax01:40', 'vlan': '4095'}, {'port_id': 'urn:sdx:port:sax.net:Sax01:41', 'vlan': '4091'}, {'port_id': 'urn:sdx:port:tenet.ac.za:Tenet01:41', 'vlan': '4091'}, {'port_id': 'urn:sdx:port:tenet.ac.za:Tenet03:50', 'vlan': '4095'}]",None
ba1db993-b1b1-4f25-99bb-57d6cba7c87a,demo 1,"[{'port_id': 'urn:sdx:port:ampath.net:Ampath3:50', 'vlan': '666'}]",None,None,None,None,None,None,None,None,None,None,"[{'port_id': 'urn:sdx:port:ampath.net:Ampath3:50', 'vlan': '666'}, {'port_id': 'urn:sdx:port:ampath.net:Ampath1:40', 'vlan': '4092'}, {'port_id': 'urn:sdx:port:sax.net:Sax01:40', 'vlan': '4090'}, {'port_id': 'urn:sdx:port:sax.net:Sax01:40', 'vlan': '4089'}]",None
cc9f077b-3ed1-41ef-a762-d8b992b83638,demo 1,"[{'port_id': 'urn:sdx:port:ampath.net:Ampath3:50', 'vlan': '4093'}]",None,None,None,None,None,None,None,None,None,None,"[{'port_id': 'urn:sdx:port:ampath.net:Ampath3:50', 'vlan': '4093'}, {'port_id': 'urn:sdx:port:ampath.net:Ampath1:40', 'vlan': '4093'}, {'port_id': 'urn:sdx:port:sax.net:Sax01:40', 'vlan': '4092'}, {'port_id': 'urn:sdx:port:sax.net:Sax01:40', 'vlan': '4091'}]",None
da5e7cc3-57f6-4e05-b5f4-575be256047a,demo 1,"[{'port_id': 'urn:sdx:port:ampath.net:Ampath3:50', 'vlan': '1312'}, {'port_id': 'urn:sdx:port:sax.net:Sax02:50', 'vlan': '1312'}]",None,None,None,None,None,None,None,None,None,None,"[{'port_id': 'urn:sdx:port:ampath.net:Ampath3:50', 'vlan': '1312'}, {'port_id': 'urn:sdx:port:am

In [12]:
l2vpn_data = client.get_l2vpn(service_id='feacaae7-f116-4594-8060-a530f3640d8b')

node_info_list = []

for service_id, service_details in l2vpn_data.items():
    service_dict = {}
    for key, value in service_details.items():
        service_dict[key] = value
    node_info_list.append(service_dict)

df = pd.DataFrame(node_info_list)
df.style.hide(axis="index")

service_id,name,endpoints,description,qos_metrics,notifications,ownership,creation_date,archived_date,status,state,counters_location,last_modified,current_path,oxp_service_ids
feacaae7-f116-4594-8060-a530f3640d8b,demo 1,"[{'port_id': 'urn:sdx:port:ampath.net:Ampath3:50', 'vlan': '1313'}, {'port_id': 'urn:sdx:port:sax.net:Sax01:50', 'vlan': '1313'}]",None,None,None,None,None,None,None,None,None,None,"[{'port_id': 'urn:sdx:port:ampath.net:Ampath3:50', 'vlan': '1313'}, {'port_id': 'urn:sdx:port:ampath.net:Ampath1:40', 'vlan': '4091'}, {'port_id': 'urn:sdx:port:sax.net:Sax01:40', 'vlan': '4088'}, {'port_id': 'urn:sdx:port:sax.net:Sax01:50', 'vlan': '1313'}]",None


In [13]:
delete_list = []
for service_id,item in l2vpn_data.items():
    if item['name'] == "Test SDX SC24 L2VPN":
        print(service_id)
        print(item['name'])
        delete_list.append(service_id)

In [14]:
for client.service_id in delete_list:
    print (client.service_id)
    client.delete_l2vpn(service_id = client.service_id)

In [33]:
# This one should pass

In [34]:
stdout, stderr = node1.execute(f'ping -c 5 192.168.1.10')

PING 192.168.1.10 (192.168.1.10) 56(84) bytes of data.
From 192.168.1.101 icmp_seq=1 Destination Host Unreachable
From 192.168.1.101 icmp_seq=2 Destination Host Unreachable
From 192.168.1.101 icmp_seq=3 Destination Host Unreachable
From 192.168.1.101 icmp_seq=4 Destination Host Unreachable
From 192.168.1.101 icmp_seq=5 Destination Host Unreachable

--- 192.168.1.10 ping statistics ---
5 packets transmitted, 0 received, +5 errors, 100% packet loss, time 4074ms
pipe 3


## Delete the Slice

Please delete your slice when you are done with your experiment.

In [35]:
slice.delete()